In [1]:
import sys
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image as im
from scipy.ndimage import interpolation as inter
import cv2
from imutils.perspective import four_point_transform

In [2]:
image = cv2.imread("test2.jpeg")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# cv2.imshow("gray", gray)

# input_file = "test2.jpeg"
# img = im.open(input_file)
# cv2.imshow()
# # imgf contains Binary image
# # ret, imgf = cv2.threshold(img, 0, 255,cv2.THRESH_BINARY,cv2.THRESH_OTSU)
# # convert to binary
# wd, ht = img.size
# pix = np.array(img.convert("1").getdata(), np.uint8)
# wd, ht

In [3]:
blur = cv2.GaussianBlur(gray, (7,7), 0)
# cv2.imshow("blur", blur)

In [5]:
thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
cv2.imshow("thresh", thresh)

In [ ]:
# Find contours and sort for largest contour
cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
displayCnt = None

for c in cnts:
    # Perform contour approximation
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)
    if len(approx) == 4:
        displayCnt = approx
        break

# Obtain birds' eye view of image
warped = four_point_transform(image, displayCnt.reshape(4, 2))

# cv2.imshow("thresh", thresh)
cv2.imshow("warped", warped)
cv2.waitKey()

In [ ]:
bin_img = 1 - (pix.reshape((ht, wd)) / 255.0)
plt.imshow(bin_img, cmap="gray")
plt.savefig("binary.png")

In [ ]:
def find_score(arr, angle):
    data = inter.rotate(arr, angle, reshape=False, order=0)
    hist = np.sum(data, axis=1)
    score = np.sum((hist[1:] - hist[:-1]) ** 2)
    return hist, score

In [ ]:
delta = 1
limit = 5
angles = np.arange(-limit, limit + delta, delta)
scores = []
for angle in angles:
    hist, score = find_score(bin_img, angle)
    scores.append(score)
best_score = max(scores)
best_angle = angles[scores.index(best_score)]
print("Best angle: {}".format(best_angle))

In [ ]:
# correct skew
data = inter.rotate(bin_img, best_angle, reshape=False, order=0)
img = im.fromarray((255 * data).astype("uint8")).convert("RGB")
# img
img.save("skew_corrected.png")

# dst = cv2.fastNlMeansDenoisingColored(img, None, 10, 10, 7, 15)
# Plotting of source and destination image
plt.subplot(121), plt.imshow(img)
# plt.subplot(122), plt.imshow(dst)
plt.show()